In [4]:
import cv2
import numpy as np
import rasterio

# Read the TIFF file
image = cv2.imread("./VanDijk.tif")

# Convert image to RGB color space
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Define lower and upper bounds for red color
lower_red = np.array([200, 70, 0])
upper_red = np.array([255, 120, 70])

# Threshold the image to get regions of red color
mask = cv2.inRange(image_rgb, lower_red, upper_red)

# Optionally, perform edge detection
edges = cv2.Canny(mask, 50, 150)

# Get spatial information from the input image
with rasterio.open("./VanDijk.tif") as src:
    profile = src.profile

# Save the detected lines as a TIFF file with the same spatial information
with rasterio.open('detected_lines2.tif', 'w', **profile) as dst:
    dst.write(edges, 1)

In [7]:
import rasterio
from rasterio.features import shapes
import fiona
from shapely.geometry import shape, LineString, Polygon

# Read the detected_lines.tif file
with rasterio.open('detected_lines2.tif') as src:
    # Read raster data as numpy array
    data = src.read(1)

    # Find the pixels with the specified RGB value (255, 0, 0)
    indices = (data == 255)

    # Get the shapes (contours) of the detected lines
    results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) in enumerate(
            shapes(data, mask=indices, transform=src.transform)))

    # Convert pixel coordinates to spatial coordinates
    shapes_geo = []
    for result in results:
        geom = shape(result['geometry'])
        # If the geometry is a polygon, convert it to a linestring
        if geom.geom_type == 'Polygon':
            geom = LineString(geom.exterior.coords)
        # Transform geometry from pixel coordinates to spatial coordinates
        geom_geo = rasterio.warp.transform_geom(src.crs, 'EPSG:4326', geom, precision=6)
        shapes_geo.append(geom_geo)

# Save the detected lines as a shapefile
schema = {'geometry': 'LineString', 'properties': {'raster_val': 'int'}}
with fiona.open('detected_lines2.shp', 'w', 'ESRI Shapefile', schema) as c:
    for geom in shapes_geo:
        c.write({
            'geometry': geom,
            'properties': {'raster_val': 255}  # Assuming the value is always 255
        })

print("Shapefile saved successfully.")


Shapefile saved successfully.
